<a href="https://colab.research.google.com/github/vasudevgupta7/cs4830-assignments/blob/main/lab-8/ME18B182_lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Dependencies and Import Required Packages

In [ ]:
%%capture

# !sudo add-apt-repository ppa:openjdk-r/ppa

!sudo apt-get install openjdk-11-jdk
# To Install Oracke JDK varsion 8
# !sudo add-apt-repository ppa:webupd8team/java
# !sudo apt-get install oracle-java8-installer

In [ ]:
%%capture

# !wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
# !tar xvzf spark-3.1.1-bin-hadoop3.2.tgz
!pip install pyspark
!pip install -q findspark
!pip install pyarrow
try:
  # %tensorflow_version only exists in Colab.
  !pip install  tf-estimator-nightly==2.8.0.dev2021122109
except Exception:
  print("skipping tf-estimator-nightly==2.8.0.dev2021122109")
  pass

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1"


In [4]:
from pyspark.sql.functions import col, pandas_udf, regexp_extract
import io

from tensorflow.keras.applications.imagenet_utils import decode_predictions
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, PandasUDFType

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pathlib
import findspark
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt 


findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [19]:
%%capture
!wget https://www.cs.toronto.edu/%7Ekriz/cifar-10-python.tar.gz
!tar xvf cifar-10-python.tar.gz

--2022-04-16 04:46:21--  https://www.cs.toronto.edu/%7Ekriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M   101MB/s    in 1.6s    

2022-04-16 04:46:23 (101 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1
batches.meta  data_batch_2  data_batch_4  readme.html
data_batch_1  data_batch_3  data_batch_5  test_batch


In [ ]:
!ls cifar-10-batches-py/

In [20]:
import pickle

def load_cifar10_batch(cifar10_dataset_folder_path, batch_id):

  with open(cifar10_dataset_folder_path + '/data_batch_' + str(batch_id), mode='rb') as file:
      # note the encoding type is 'latin1'
      batch = pickle.load(file, encoding='latin1')
      
  features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
  labels = batch['labels']
      
  return features, labels

def load_label_names():
  return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [21]:
import numpy as np

features_p = np.array([])
labels_p = np.array([])

for batch_id in range(1,6):
  features, labels = load_cifar10_batch('./cifar-10-batches-py', batch_id)
  labels = np.expand_dims(np.squeeze(labels),1)
  if batch_id-1:
    features_acc = np.vstack([features_p, features])
    labels_acc = np.vstack([labels_p, labels])
    features_p = features_acc
    labels_p = labels_acc
  else:
    features_p = features
    labels_p = labels

In [22]:
label_names = load_label_names()
features_acc.shape, labels_acc.shape

((50000, 32, 32, 3), (50000, 1))

In [23]:
from PIL import Image as im
import os
import shutil
from tqdm.notebook import tqdm

def write_imagenet_format(features_acc, labels_acc, data_path):
  label_names = load_label_names()

  if not os.path.exists(data_path):
    os.makedirs(data_path)

  for label in label_names:
    sub_fold = os.path.join(data_path,label)
    if not os.path.exists(sub_fold):
      os.mkdir(sub_fold)

  for i in tqdm(range(features_acc.shape[0])):
    samp = features_acc[i]
    label = np.squeeze(labels_acc[i])
    data = im.fromarray(samp, 'RGB')
    data_save_path = os.path.join(data_path,label_names[label],str(i)+'.jpg')
    data.save(data_save_path)

In [24]:
train_path = 'c10_data/train_data'
write_imagenet_format(features_acc, labels_acc, train_path)

  0%|          | 0/50000 [00:00<?, ?it/s]

In [26]:
import pickle
with open('./cifar-10-batches-py/test_batch', mode='rb') as file:
    batch = pickle.load(file, encoding='latin1')
test_features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
test_labels = batch['labels']

In [27]:
test_path = 'c10_data/test_data'
write_imagenet_format(test_features, test_labels, test_path)

  0%|          | 0/10000 [00:00<?, ?it/s]

Download the Public Dataset

In [28]:
# data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
#                                          fname='flower_photos', untar=True)

data_dir = './c10_data/train_data'
print(data_dir)

./c10_data/train_data


Recursively Read all the Images from the Parent directory and create a DataFrame

In [29]:
images = spark.read.format("binaryFile").option("recursiveFileLookup", "true").option("pathGlobFilter", "*.jpg").load(data_dir)
print(type(images))

<class 'pyspark.sql.dataframe.DataFrame'>


In [30]:
images.show(5)

+--------------------+--------------------+------+--------------------+
|                path|    modificationTime|length|             content|
+--------------------+--------------------+------+--------------------+
|file:/content/c10...|2022-04-16 04:47:...|  1124|[FF D8 FF E0 00 1...|
|file:/content/c10...|2022-04-16 04:47:...|  1121|[FF D8 FF E0 00 1...|
|file:/content/c10...|2022-04-16 04:47:...|  1118|[FF D8 FF E0 00 1...|
|file:/content/c10...|2022-04-16 04:47:...|  1111|[FF D8 FF E0 00 1...|
|file:/content/c10...|2022-04-16 04:47:...|  1104|[FF D8 FF E0 00 1...|
+--------------------+--------------------+------+--------------------+
only showing top 5 rows



In [ ]:
# files=images.select('path').rdd.map(lambda x :x.path ).collect()
# files[0:10]

In [32]:
regexp = "./c10_data/train_data/([^/]+)"

def extract_label(path_col):
  """Extract label from file path using built-in SQL functions."""
  return regexp_extract(path_col, regexp, 1)

def extract_size(content):
  """Extract image size from its raw content."""
  image = Image.open(io.BytesIO(content))
  return image.size

@pandas_udf("width: int, height: int")
def extract_size_udf(content_series):
  sizes = content_series.apply(extract_size)
  return pd.DataFrame(list(sizes))

df = images.select(
  col("path"),
  col("modificationTime"),
  extract_label(col("path")).alias("label"),
  extract_size_udf(col("content")).alias("size"),
  col("content"))


# df.show(5)


In [33]:
ima=images.select('content').rdd.map(lambda x: x.content).collect()
imgg = Image.open(io.BytesIO(ima[0]))
imgg

bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x00 \x00 \x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\

In [36]:
class ImageNetDataset(Dataset):
  """
  Converts image contents into a PyTorch Dataset with standard ImageNet preprocessing.
  """
  def __init__(self, contents):
    self.contents = contents

  def __len__(self):
    
    return len(self.contents)

  def __getitem__(self, index):
    return self._preprocess(self.contents[index])

  def _preprocess(self, content):
    """
    Preprocesses the input image content using standard ImageNet normalization.
    
    See https://pytorch.org/docs/stable/torchvision/models.html.
    """
    image = Image.open(io.BytesIO(content))
    transform = transforms.Compose([
      transforms.Resize(256),                         ### Smaller lenght is convertd to 256 
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image)

In [37]:

def imagenet_model_udf(model):
  """
  Wraps an ImageNet model into a Pandas UDF that makes predictions.
  
  You might consider the following customizations for your own use case:
    - Tune DataLoader's batch_size and num_workers for better performance.
    - Use GPU for acceleration.
    - Change prediction types.
  """
  model.eval()
  def predict(content_series_iter : pd.Series) -> pd.DataFrame:
    for content_series in content_series_iter:                                                    #Iterates overall all Images 
      dataset = ImageNetDataset(list(content_series))                                           
      loader = DataLoader(dataset, batch_size=64) 
      with torch.no_grad():
        for image_batch in loader:
          predictions = model(image_batch).numpy()                                                # Predictions for all 1000 classes of Mobilenetv2 Training Dataset
          predicted_labels = [x[0] for x in decode_predictions(predictions, top=1)]                       
          yield pd.DataFrame(predicted_labels)

  return_type = "class: string, desc: string, score:float"
  # return_type = "class: string"                                         # SCALAR_ITER implies the function accepts list/array of scalar values as input.
  return pandas_udf(predict,return_type, PandasUDFType.SCALAR_ITER)   
  # return pandas_udf(return_type, PandasUDFType.SCALAR_ITER)(predict)   
  
  #Returns a Predict function object as the final user defined function object.
  #Iterates over an Image: A sequence of bytes represented as a 2D array. 
  #Scalar is a series of bytes.    

In [38]:
# imagenet = imagenet_model_udf(lambda: models.mobilenet_v2(pretrained=True))

model=models.mobilenet_v2(pretrained=True)
imagenet = imagenet_model_udf(model)
imagenet

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  "in the future releases. See SPARK-28264 for more details.", UserWarning)


<function __main__.imagenet_model_udf.<locals>.predict>

In [39]:
predictions = df.withColumn("prediction", imagenet(col("content")))
display(predictions.select(col("path"), col("prediction")).limit(5))

DataFrame[path: string, prediction: struct<class:string,desc:string,score:float>]

In [40]:
predictions.select(col("label"),col("prediction")).show(50,truncate=False)

+----------+---------------------------------------+
|label     |prediction                             |
+----------+---------------------------------------+
|frog      |{n01744401, rock_python, 7.9679623}    |
|bird      |{n03944341, pinwheel, 10.355642}       |
|truck     |{n02817516, bearskin, 10.859891}       |
|automobile|{n03794056, mousetrap, 12.167225}      |
|truck     |{n03843555, oil_filter, 9.178589}      |
|truck     |{n04428191, thresher, 17.19516}        |
|frog      |{n02128925, jaguar, 11.967956}         |
|truck     |{n03796401, moving_van, 11.574023}     |
|airplane  |{n04542943, waffle_iron, 11.026524}    |
|automobile|{n03884397, panpipe, 11.6462755}       |
|frog      |{n01756291, sidewinder, 11.004475}     |
|truck     |{n02690373, airliner, 9.261075}        |
|automobile|{n03720891, maraca, 7.206638}          |
|truck     |{n04428191, thresher, 12.286238}       |
|frog      |{n03047690, clog, 9.803439}            |
|truck     |{n04428191, thresher, 13.313608}  